# Carlos Code

In [ ]:
from classy import Class
# define a theano Op for our likelihood function
class bao_lkl(tt.Op):

    """
    Specify what type of object will be passed and returned to the Op when it is
    called. In our case we will be passing it a vector of values (the parameters
    that define our model) and returning a single "scalar" value (the
    log-likelihood)
    """
    itypes = [tt.dvector] # expects a vector of parameter values when called
    otypes = [tt.dscalar] # outputs a single scalar value (the log likelihood)

    def __init__(self):
        """
        Initialise the Op with various things that our log-likelihood function
        requires. Below are the things that are needed in this particular
        example.

        Parameters
        ----------
        """
            
        # Data from montepython_public/data/COMBINEDDR12_BAO_consensus_dM_Hz
        self.rsfid = 147.78
        cov = np.array([
            [624.707, 23.729, 325.332, 8.34963, 157.386, 3.57778],
            [23.729, 5.60873, 11.6429, 2.33996, 6.39263, 0.968056],
            [325.332, 11.6429, 905.777, 29.3392, 515.271, 14.1013],
            [8.34963, 2.33996, 29.3392, 5.42327, 16.1422, 2.85334],
            [157.386, 6.39263, 515.271, 16.1422, 1375.12, 40.4327],
            [3.57778, 0.968056, 14.1013, 2.85334, 40.4327, 6.25936]
        ])
        self.icov = np.linalg.inv(cov)
        # BAO-only consensus results, Alam et al. 2016
        self.z = np.array([0.38, 0.51, 0.61])
        self.a = 1/(1+self.z)
        # Vectors are multiplied by (rsfid/rs) so that: dM = dM*(rsfid/rs) 
        # and Hz = Hz*(rs/rsfid)
        dM = np.array([1512.39, 1975.22, 2306.68])
        Hz = np.array([81.2087, 90.9029, 98.9647])

        data_vector = np.empty((dM.size + Hz.size), dtype=dM.dtype)
        data_vector[0::2] = dM
        data_vector[1::2] = Hz
        
        self.data = data_vector
        self.model = Class()

    def likelihood(self, theta):
        if np.any(theta < 0):
            return -np.inf

        Omega_c, h = theta
#         cosmo = ccl.Cosmology(Omega_c=Omega_c, Omega_b=0.045, h=h, sigma8=0.78, n_s=0.96,
#                                transfer_function='boltzmann_class')
#         Hz = ccl.background.h_over_h0(cosmo, self.a) * h * 100
#         dM = ccl.background.comoving_angular_distance(cosmo, self.a)
        
#         params = {
#         "h": cosmo["h"],
#         "Omega_cdm": cosmo["Omega_c"],
#         "Omega_b": cosmo["Omega_b"],
#         "Omega_k": cosmo["Omega_k"],
#         "n_s": cosmo["n_s"],
#         "T_cmb": cosmo['T_CMB']}
    
        params = {'h': h,
                  'Omega_cdm': Omega_c}
        
        model = self.model
        model.set(params)
        try:
            model.compute()
        except:
            model.struct_cleanup()
            return -np.inf
        rs = model.rs_drag()
        dM = np.array([model.angular_distance(zi) * (1. + zi) for zi in self.z])
        Hz = np.array([model.Hubble(zi) for zi in self.z]) * 2.99792458e8 / 1000.0
        model.struct_cleanup()
        model.empty()
                
        # Not sure how to compute r_s in CCL. Not included
        th_vector = np.empty((dM.size + Hz.size), dtype=dM.dtype)
        th_vector[0::2] = dM * self.rsfid / rs
        th_vector[1::2] = Hz * rs / self.rsfid
        
        logl = -0.5 * (th_vector - self.data).dot(self.icov).dot(th_vector - self.data)
        return logl

    def perform(self, node, inputs, outputs):
        # the method that is used when calling the Op
        theta, = inputs  # this will contain my variables

        # call the log-likelihood function
        logl = self.likelihood(theta)

        outputs[0][0] = np.array(logl) # output the log-likelihood
    

logl = bao_lkl()
    
with pm.Model():
    # uniform priors on m and c
    Omega_c = pm.Normal('Omega_c', mu=0.26, sigma=0.25)
    h = pm.Normal('h', mu=0.68, sigma=0.25)

    # convert m and c to a tensor vector
    theta = tt.as_tensor_variable([Omega_c, h])

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.Potential('likelihood', logl(theta))
    #pm.Potential('likelihood', lambda v: logl(v), observed={'v': theta})

    step = pm.Metropolis()
    #trace = pm.sample(100, step=step, cores=2)